1. Data Preprocessing

In [25]:
import pandas as pd
import nltk
import re
import numpy as np
import math
from collections import Counter
from typing import List, Sequence, Tuple
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [26]:
nltk.download("stopwords")
nltk.download("punkt")

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
df = pd.read_csv("bbc-text.csv")
df

category  \
0              tech   
1          business   
2             sport   
3             sport   
4     entertainment   
...             ...   
2220       business   
2221       politics   
2222  entertainment   
2223       politics   
2224          sport   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [28]:
def clean_and_tokenize(text: str) -> List[str]:
    if not isinstance(text, str):
        return []
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    tokens = text.split()
    tokens = [tok for tok in tokens if tok not in stop_words and len(tok) > 1]
    return tokens
df["tokens"] = df["text"].apply(clean_and_tokenize)

In [29]:
df

category  \
0              tech   
1          business   
2             sport   
3             sport   
4     entertainment   
...             ...   
2220       business   
2221       politics   
2222  entertainment   
2223       politics   
2224          sport   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [30]:
vocab_set = set()
for toks in df["tokens"]:
    vocab_set.update(toks)

vocab_list = sorted(vocab_set)
vocab_index = {w: i for i, w in enumerate(vocab_list)}
print("Vocabulary size:", len(vocab_list))
print("First 30 words:", vocab_list[:30])

Vocabulary size: 29280
First 30 words: ['00', '000', '0001', '000bn', '000m', '000s', '000th', '001', '001and', '001st', '004', '0051', '007', '01', '0100', '011', '0130', '02', '0200', '0227', '028', '03', '0300', '033', '037', '03bn', '04', '0400', '041', '04bn']


2. Implement Bag of Words

In [31]:
bow = []
for doc_tokens in df["tokens"]:
    vec = [0] * len(vocab_list)
    for t in doc_tokens:
        vec[vocab_index[t]] += 1
    bow.append(vec)
bow = np.array(bow)
print("BoW shape:", bow.shape)

BoW shape: (2225, 29280)


3.TF-IDF

In [32]:
def compute_tf(document_tokens: Sequence[str], vocabulary: Sequence[str]) -> List[float]:
    doc_len = len(document_tokens)
    counts = Counter(document_tokens)
    if doc_len == 0:
        return [0.0 for _ in vocabulary]
    return [counts.get(word, 0) / doc_len for word in vocabulary]

In [33]:
def compute_idf(all_docs: Sequence[Sequence[str]], vocabulary: Sequence[str]) -> List[float]:
    N = len(all_docs)
    idf_vector: List[float] = []
    for word in vocabulary:
        df_count = sum(1 for doc in all_docs if word in doc)
        idf = math.log(N / (df_count + 1)) + 1
        idf_vector.append(idf)
    return idf_vector


In [34]:
def compute_tfidf(doc_tokens: Sequence[str],
                  vocabulary: Sequence[str],
                  idf_vector: Sequence[float]) -> List[float]:
    tf_vector = compute_tf(doc_tokens, vocabulary)
    return [tf * idf for tf, idf in zip(tf_vector, idf_vector)]

In [35]:
def top_k_tfidf(tfidf_vector: Sequence[float],
                vocabulary: Sequence[str],
                k: int = 10) -> List[Tuple[str, float]]:
    pairs = [(w, s) for w, s in zip(vocabulary, tfidf_vector) if s > 0]
    pairs.sort(key=lambda x: x[1], reverse=True)
    return pairs[:k]

In [36]:
all_docs_tokens = df["tokens"].tolist()
idf_vector = compute_idf(all_docs_tokens, vocab_list)
tfidf_matrix = [compute_tfidf(doc, vocab_list, idf_vector) for doc in all_docs_tokens]


In [37]:
print("First training doc TF-IDF vector sample:", tfidf_matrix[0][:50], "...")
print("Top words in first doc:", top_k_tfidf(tfidf_matrix[0], vocab_list, k=10))

First training doc TF-IDF vector sample: [0.0, 0.0059839621651252995, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ...
Top words in first doc: [('tv', 0.10466056676919147), ('dvr', 0.07229358580458177), ('hanlon', 0.05710948941121422), ('definition', 0.04744836226029485), ('tivo', 0.04639159822278471), ('want', 0.045666384762089), ('watch', 0.04196724706367649), ('satellite', 0.041067030038721385), ('content', 0.03907261134884323), ('brands', 0.03692032068872337)]


4. Analysis

In [38]:
tfidf_arr = np.array(tfidf_matrix)
tf_matrix = np.array([compute_tf(toks, vocab_list) for toks in df["tokens"]])

mean_tf = tf_matrix.mean(axis=0)
idf_arr = np.array(idf_vector)

words_df = pd.DataFrame({
    "word": vocab_list,
    "mean_tf": mean_tf,
    "idf": idf_arr
})

In [39]:
def top_words_by_category(category_name: str, top_k: int = 10):
    if "category" in df.columns:
        col = "category"
    elif "label" in df.columns:
        col = "label"
    else:
        print("No category:", df.columns.tolist())
        return

    mask = df[col] == category_name
    if mask.sum() == 0:
        print(f"No documents found for category: {category_name!r}")
        return

    avg_tfidf = tfidf_arr[mask.values].mean(axis=0)
    top_words = top_k_tfidf(avg_tfidf.tolist(), vocab_list, k=top_k)

    print(f"\nTop {top_k} words for category '{category_name}':")
    for rank, (w, score) in enumerate(top_words, start=1):
        print(f"{rank:2d}. {w:20s}  {score:.6f}")

In [40]:
top_by_mean_tf = words_df.sort_values("mean_tf", ascending=False).head(200)
high_tf_low_idf = top_by_mean_tf.sort_values("idf", ascending=True).head(20)

In [41]:
top_by_idf = words_df.sort_values("idf", ascending=False).head(200)
high_idf_low_tf = top_by_idf.sort_values("mean_tf", ascending=True).head(20)

In [42]:
print("\nWords with HIGH mean TF but LOW IDF (likely high-frequency 'common' words within docs):")
print(high_tf_low_idf[["word", "mean_tf", "idf"]].to_string(index=False))


Words with HIGH mean TF but LOW IDF (likely high-frequency 'common' words within docs):
  word  mean_tf      idf
  said 0.015108 1.163709
  also 0.004434 1.563895
  year 0.005293 1.624964
 would 0.004893 1.662607
   one 0.003672 1.730231
   new 0.003855 1.817921
  last 0.003149 1.887496
 could 0.002854 1.932146
   two 0.002755 1.945939
    us 0.004209 1.968176
  time 0.002605 1.989707
 first 0.002892 2.014189
people 0.003433 2.024151
    mr 0.005408 2.034214
 years 0.002011 2.197254
 world 0.002508 2.201728
  made 0.001782 2.227468
  make 0.001784 2.250743
  told 0.001814 2.281024
 three 0.001793 2.305595


In [43]:
print("\nWords with HIGH IDF but LOW mean TF (rare words overall but low average frequency):")
print(high_idf_low_tf[["word", "mean_tf", "idf"]].to_string(index=False))


Words with HIGH IDF but LOW mean TF (rare words overall but low average frequency):
      word      mean_tf      idf
        lp 2.455946e-07 8.014365
     yoffi 3.536099e-07 8.014365
yesteryear 3.536099e-07 8.014365
     loops 5.507821e-07 8.014365
    louder 6.628882e-07 8.014365
     lucid 7.168073e-07 8.014365
   lorries 8.025682e-07 8.014365
      yuji 8.626453e-07 8.014365
       127 8.626453e-07 8.014365
      100s 8.760979e-07 8.014365
      000s 8.760979e-07 8.014365
 loudmouth 9.305139e-07 8.014365
  lowlifes 9.382843e-07 8.014365
     106cm 9.791682e-07 8.014365
  youthful 9.877763e-07 8.014365
        ym 1.005455e-06 8.014365
       yle 1.040366e-06 8.014365
    zander 1.040366e-06 8.014365
  lorraine 1.085600e-06 8.014365
      1606 1.085600e-06 8.014365
